# Outlining some pre-processing

In [ ]:
%load_ext jupyter_black
import os
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [ ]:
# gis_dir = Path(os.getenv("STORM_DATA_DIR")) / "gis files/"
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/input/"
)
output_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/output/"
)
adm3_shp = gpd.read_file(input_dir / "phl_adminboundaries_candidate_adm3.zip")

In [ ]:
phl_north_bld = gpd.read_file(
    input_dir / "hotosm_phl_north_buildings_polygons_shp.zip"
)
phl_south_bld = gpd.read_file(
    input_dir / "hotosm_phl_south_buildings_polygons_shp.zip"
)

In [ ]:
phl_bld = pd.concat([phl_north_bld, phl_south_bld])

In [ ]:
adm3_shp.loc[:, "geometry"].plot()

## Creation of grid

In [ ]:
# grid creation
xmin, xmax, ymin, ymax = 114.25, 126.75, 4.55, 21.15

cell_size = 0.1

cols = list(np.arange(xmin, xmax + cell_size, cell_size))
rows = list(np.arange(ymin, ymax + cell_size, cell_size))
rows.reverse()

polygons = []
for x in cols:
    for y in rows:
        polygons.append(
            Polygon(
                [
                    (x, y),
                    (x + cell_size, y),
                    (x + cell_size, y - cell_size),
                    (x, y - cell_size),
                ]
            )
        )

grid = gpd.GeoDataFrame({"geometry": polygons}, crs=adm3_shp.crs)
grid["id"] = grid.index + 1
# grid.to_crs(epsg=4326, inplace=True)
grid.head()

In [ ]:
grid.loc[:, "geometry"].plot()

In [ ]:
# write as geopackage
grid.to_file(output_dir / "phl_0.1_degree_grid.gpkg", driver="GPKG")

## Creation of centroids

In [ ]:
# creating centroids
# Grid
# Extract lat and lon from the centerpoint
grid["Longitude"] = grid["geometry"].centroid.map(lambda p: p.x)
grid["Latitude"] = grid["geometry"].centroid.map(lambda p: p.y)
grid["Centroid"] = (
    round(grid["Longitude"], 2).astype(str)
    + "E"
    + "_"
    + +round(grid["Latitude"], 2).astype(str)
    + "N"
)
grid.head(5)

In [ ]:
# Centroids
grid_centroids = grid.copy()
grid_centroids["geometry"] = grid_centroids["geometry"].centroid
grid_centroids.loc[:, "geometry"].plot()

In [ ]:
# write as geopackage
grid_centroids.to_file(
    output_dir / "phl_0.1_degree_grid_centroids.gpkg", driver="GPKG"
)

In [ ]:
# building centroids
phl_bld_centroids = phl_bld.copy()
phl_bld_centroids["geometry"] = phl_bld_centroids["geometry"].centroid
phl_bld_centroids.to_file(input_dir / "phl_bld_centroids.gpkg", driver="GPKG")

## Intersection of grid and admin shapefile

In [ ]:
# intersection of grid and shapefile
adm3_grid_intersection = gpd.overlay(adm3_shp, grid, how="identity")
# adm3_grid_intersection = gpd.sjoin(adm3_shp, grid)
adm3_grid_intersection.loc[:, "geometry"].plot()
plt.axis([124, 126, 10, 11])

In [ ]:
adm3_grid_intersection = adm3_grid_intersection.dropna(subset=["id"])
adm3_grid_intersection.loc[
    adm3_grid_intersection["ADM3_PCODE"] == "PH084808000"
]

### For only grids that overlap with land

In [ ]:
grid_land_overlap = grid.loc[grid["id"].isin(adm3_grid_intersection["id"])]
grid_land_overlap.to_file(
    output_dir / "phl_0.1_degree_grid_land_overlap.gpkg", driver="GPKG"
)
grid_land_overlap_centroids = grid_centroids.loc[
    grid["id"].isin(adm3_grid_intersection["id"])
]
grid_land_overlap_centroids.to_file(
    output_dir / "phl_0.1_degree_grid_centroids_land_overlap.gpkg",
    driver="GPKG",
)

## Area Computation

In [ ]:
# computing areas
adm3_shp_area = adm3_shp.copy()
adm3_shp_area = adm3_shp_area.to_crs({"proj": "cea"})
adm3_shp_area["Area"] = adm3_shp_area["geometry"].area
adm3_shp_area["Area"].describe()
adm3_shp_area.to_csv(input_dir / "adm3_area.csv")

In [ ]:
grid_area = grid.copy()
grid_area = grid_area.to_crs({"proj": "cea"})
grid_area["Area_2"] = grid_area["geometry"].area
grid_area["Area_2"].describe()

In [ ]:
adm3_grid_intersection_area = adm3_grid_intersection.copy()
adm3_grid_intersection_area = adm3_grid_intersection_area.to_crs(
    {"proj": "cea"}
)
adm3_grid_intersection_area["Area_3"] = adm3_grid_intersection_area[
    "geometry"
].area
adm3_grid_intersection_area["Area_3"].describe()

In [ ]:
# merge
phl_adm3_grid_intersections_area = adm3_shp_area.merge(
    grid_area.merge(
        adm3_grid_intersection_area, on="id", suffixes=(None, "_3")
    ),
    on="ADM3_PCODE",
    suffixes=(None, "_2"),
)

phl_adm3_grid_intersections_area.drop(
    phl_adm3_grid_intersections_area.filter(regex="geometry").columns,
    axis=1,
    inplace=True,
)
phl_adm3_grid_intersections_area.to_csv(
    input_dir / "phl_adm3_grid_intersections_area.csv"
)

## Building Count

### Assigning Buildings to Grids

In [ ]:
phl_bld_centroid_grid_count = grid_land_overlap.merge(
    grid_land_overlap.sjoin(phl_bld_centroids)
    .groupby("id")
    .count()
    .geometry.rename("numbuildings")
    .reset_index()
)
phl_bld_centroid_grid_count.to_csv(
    input_dir / "phl_hotosm_bld_centroid_grid_count.csv"
)

In [ ]:
phl_bld_centroid_grid_count.head(8)

In [ ]:
phl_bld_centroid_grid_count["numbuildings"].sum()

### Testing new way of counting buildings

In [ ]:
# assigning buildings to municipalities
phl_bld_centroid_municip_count = adm3_shp.merge(
    adm3_shp.sjoin(phl_bld_centroids)
    .groupby("ADM3_PCODE")
    .count()
    .geometry.rename("numbuildings")
    .reset_index()
)
phl_bld_centroid_municip_count.to_csv(
    input_dir / "phl_hotosm_bld_centroid_municip_count.csv"
)

In [ ]:
phl_bld_centroid_municip_count["numbuildings"].sum()

In [ ]:
# assigning buildings to intersections
phl_bld_centroid_municip_intersection_count = adm3_grid_intersection.merge(
    adm3_grid_intersection.sjoin(phl_bld_centroids)
    .groupby(["ADM3_PCODE", "Centroid"])
    .count()
    .geometry.rename("numbuildings")
    .reset_index()
)
phl_bld_centroid_municip_intersection_count.to_csv(
    input_dir / "phl_hotosm_bld_centroid_municip_intersection_count.csv"
)

In [ ]:
phl_bld_centroid_municip_count.shape

In [ ]:
phl_bld_centroid_municip_intersection_count.shape

In [ ]:
phl_bld_weight_matrix = phl_bld_centroid_municip_count.merge(
    phl_bld_centroid_municip_intersection_count,
    on="ADM3_PCODE",
    suffixes=(None, "_y"),
)
phl_bld_weight_matrix["weight"] = (
    phl_bld_weight_matrix["numbuildings_y"]
    / phl_bld_weight_matrix["numbuildings"]
)

In [ ]:
phl_bld_weight_matrix["weight"].describe()

In [ ]:
phl_bld_weight_matrix.columns

In [ ]:
phl_bld_weight_matrix[
    ["ADM3_PCODE", "ADM3_EN", "id", "Centroid", "weight"]
].to_csv(input_dir / "phl_bld_weight_matrix.csv")